Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

---

In [1]:
from random import randrange
import numpy as np
from sklearn.metrics import mean_squared_error, log_loss
from sklearn.datasets import load_breast_cancer, load_diabetes
from sklearn.linear_model import Lasso, LinearRegression
from sklearn.preprocessing import normalize


def grad_check_sparse(f, x, analytic_grad, num_checks=10, h=1e-5, error=1e-9):
    """
    sample a few random elements and only return numerical
    in this dimensions
    """

    for i in range(num_checks):
        ix = tuple([randrange(m) for m in x.shape])

        oldval = x[ix]
        x[ix] = oldval + h  # increment by h
        fxph = f(x)  # evaluate f(x + h)
        x[ix] = oldval - h  # increment by h
        fxmh = f(x)  # evaluate f(x - h)
        x[ix] = oldval  # reset

        grad_numerical = (fxph - fxmh) / (2 * h)
        grad_analytic = analytic_grad[ix]
        rel_error = abs(grad_numerical - grad_analytic) / (
            abs(grad_numerical) + abs(grad_analytic)
        )
        print(
            "numerical: %f analytic: %f, relative error: %e"
            % (grad_numerical, grad_analytic, rel_error)
        )
        assert rel_error < error

def rel_error(x, y):
    """ returns relative error """
    return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

In [2]:
data = load_diabetes()
X, y = data.data, data.target

In [3]:
def mse_loss_vectorized(w, b, X, y):
    """
    MSE loss function WITHOUT FOR LOOPs , NO REGULARIZATION
    
    Returns a tuple of:
    - loss 
    - gradient with respect to weights w
    - gradient with respect to bias b
    """
    loss = 0.0
    dw = np.zeros_like(w)
    
    # YOUR CODE HERE
    N = X.shape[0]
    ypred = X@w + b
    
    loss = np.mean((y-ypred)**2)
    
    dw = (-2/N)*(X.T @ (y-ypred))
    db = (-2/N)*sum(y-ypred)
    
    return loss, dw, np.array(db).reshape(1,)

In [4]:
def soft_threshold(x, delta):
    # YOUR CODE HERE
    if x>delta:
        return x-delta
    elif x<-delta:
        return x+delta
    else:
        return 0

# Lasso Subgradient Descent

In [5]:
def l1_subgradient(w):
    """
    Subgradient of the L1 loss
    """
    return np.sign(w)
    

def lasso_subgradient_mse_loss_vectorized(w, b, X, y, alpha):
    """
    MSE loss function adding the subgradient for w
    """
    loss, dw, db = mse_loss_vectorized(w, b, X, y)
    # Add the subgradient to dw
    # YOUR CODE HERE
    dw += alpha*l1_subgradient(w) 
    return loss, dw, db

In [6]:
class LassolSubgradientDescent():
    def __init__(self,  alpha=0.1):
        self.w = None
        self.b = None
        self.alpha = alpha

    def train(self, X, y, learning_rate=1e-3, num_iters=100, batch_size=200, verbose=False):
        N, d = X.shape
        
        if self.w is None: # Initialization
            self.w = 0.001 * np.random.randn(d)
            self.b = 0.0

        # Run stochastic gradient descent to optimize w
        
        loss_history = []
        for it in range(num_iters):
            X_batch = None
            y_batch = None
                                                               
            # Sample batch_size elements in X_batch and y_batch
            # X_batch shape is  (batch_size, d) and y_batch shape is (batch_size,)                                                                                          
            # Hint: Use np.random.choice to generate indices
            # YOUR CODE HERE
            choice = np.random.choice(N, batch_size, replace=False)

            X_batch = X[choice, :]
            y_batch = y[choice ]
            
            # evaluate loss and gradient
            loss, dw, db = self.loss(X_batch, y_batch)

            # perform parameter update                                                                
            # Update the weights w using the gradient and the learning rate.  
            # YOUR CODE HERE
            self.w = self.w - learning_rate * dw
            self.b = self.b - learning_rate * db
            
            if verbose and it % 10000 == 0:
                print("iteration %d / %d: loss %f" % (it, num_iters, loss))
    
    
    def predict(self, X):
        # YOUR CODE HERE
        return (X @ self.w + self.b)

    def loss(self, X_batch, y_batch):
        return lasso_subgradient_mse_loss_vectorized(self.w, self.b, X_batch, y_batch, self.alpha)

In [7]:
model = LassolSubgradientDescent(alpha=0.1)
model.train(X, y, learning_rate=1e-2,verbose=True, num_iters=200_000)
pred = model.predict(X)
mse = mean_squared_error(pred, y)

sk_model = Lasso(alpha=0.1, fit_intercept=True)
sk_model.fit(X, y)
sk_pred = sk_model.predict(X)
sk_mse = mean_squared_error(sk_pred, y)

print("MSE scikit-learn:", sk_mse)
print("MSE Coordinate descent model :", mse)
assert mse - sk_mse < 50

iteration 0 / 200000: loss 27079.737185
iteration 10000 / 200000: loss 3560.000618
iteration 20000 / 200000: loss 3075.032459
iteration 30000 / 200000: loss 3505.684795
iteration 40000 / 200000: loss 3053.193741
iteration 50000 / 200000: loss 3486.634866
iteration 60000 / 200000: loss 3037.610233
iteration 70000 / 200000: loss 3000.954199
iteration 80000 / 200000: loss 2857.151327
iteration 90000 / 200000: loss 2798.035648
iteration 100000 / 200000: loss 3128.282589
iteration 110000 / 200000: loss 3379.376540
iteration 120000 / 200000: loss 3053.053887
iteration 130000 / 200000: loss 2728.752026
iteration 140000 / 200000: loss 2725.039580
iteration 150000 / 200000: loss 3184.671060
iteration 160000 / 200000: loss 2774.477170
iteration 170000 / 200000: loss 3071.263590
iteration 180000 / 200000: loss 2824.897237
iteration 190000 / 200000: loss 2712.155639
MSE scikit-learn: 2912.521795117546
MSE Coordinate descent model : 2888.635051986439


In [8]:
model = LassolSubgradientDescent(alpha=2)
model.train(X, y, learning_rate=1e-2,verbose=True, num_iters=200_000)
pred = model.predict(X)
mse = mean_squared_error(pred, y)

sk_model = Lasso(alpha=2, fit_intercept=True)
sk_model.fit(X, y)
sk_pred = sk_model.predict(X)
sk_mse = mean_squared_error(sk_pred, y)

print("MSE scikit-learn:", sk_mse)
print("MSE Coordinate descent model :", mse)
assert mse - sk_mse < 50

iteration 0 / 200000: loss 26288.579807
iteration 10000 / 200000: loss 4207.221656
iteration 20000 / 200000: loss 4335.371506
iteration 30000 / 200000: loss 4299.475940
iteration 40000 / 200000: loss 3726.355974
iteration 50000 / 200000: loss 4266.120243
iteration 60000 / 200000: loss 3881.680747
iteration 70000 / 200000: loss 3580.931277
iteration 80000 / 200000: loss 3684.449133
iteration 90000 / 200000: loss 3718.769751
iteration 100000 / 200000: loss 4003.179601
iteration 110000 / 200000: loss 3906.304740
iteration 120000 / 200000: loss 3644.882027
iteration 130000 / 200000: loss 3939.736490
iteration 140000 / 200000: loss 3430.510325
iteration 150000 / 200000: loss 3758.821067
iteration 160000 / 200000: loss 3744.164969
iteration 170000 / 200000: loss 4097.073500
iteration 180000 / 200000: loss 3785.007231
iteration 190000 / 200000: loss 3706.250410
MSE scikit-learn: 5650.287416333697
MSE Coordinate descent model : 3813.0313841758625


# Lasso Proximal Gradient Descent (iterative soft thresholding algorithm or ISTA)

In [9]:
# source: https://www.stat.cmu.edu/~ryantibs/convexopt-S15/scribes/08-prox-grad-scribed.pdf
soft_threshold = np.vectorize(soft_threshold)
class LassoProximalGradientDescent():
    def __init__(self,  alpha=0.1):
        self.w = None
        self.b = None
        self.alpha = alpha

    def train(self, X, y, learning_rate=1e-3, num_iters=100, batch_size=200, verbose=False):
        N, d = X.shape
        
        if self.w is None: # Initialization
            self.w = 0.001 * np.random.randn(d)
            self.b = 0.0

        # Run stochastic gradient descent to optimize w
        
        loss_history = []
        for it in range(num_iters):
            X_batch = None
            y_batch = None
                                                               
            # Sample batch_size elements in X_batch and y_batch
            # X_batch shape is  (batch_size, d) and y_batch shape is (batch_size,)                                                                                          
            # Hint: Use np.random.choice to generate indices
            # YOUR CODE HEREI
            choice = np.random.choice(N, batch_size, replace=False)

            X_batch = X[choice, :]
            y_batch = y[choice ]
            
            # evaluate loss and gradient
            loss, dw, db = self.loss(X_batch, y_batch)

            # perform parameter update                                                                
            # Update the weights w using the gradient and the learning rate.  
            # PROJECT THE GRADIENT FOR w USING soft_threshold
            # YOUR CODE HERE
            self.w = self.w - soft_threshold(dw, learning_rate * self.alpha)
            self.b = self.b - learning_rate*db
            
            if verbose and it % 10000 == 0:
                print("iteration %d / %d: loss %f" % (it, num_iters, loss))

    def predict(self, X):
        # YOUR CODE HERE
        return (X@self.w + self.b)

    def loss(self, X_batch, y_batch):
        return mse_loss_vectorized(self.w, self.b, X_batch, y_batch)

In [10]:
model = LassoProximalGradientDescent(alpha=0.1)
model.train(X, y, learning_rate=1e-2,verbose=True, num_iters=200_000)
pred = model.predict(X)
mse= mean_squared_error(pred, y)

sk_model = Lasso(alpha=0.1, fit_intercept=True)
sk_model.fit(X, y)
sk_pred = sk_model.predict(X)
sk_mse = mean_squared_error(sk_pred, y)

print("MSE scikit-learn:", sk_mse)
print("MSE Coordinate descent model :", mse)
assert mse - sk_mse < 50

iteration 0 / 200000: loss 29969.478185
iteration 10000 / 200000: loss 2587.821988
iteration 20000 / 200000: loss 2843.394762
iteration 30000 / 200000: loss 2782.851254
iteration 40000 / 200000: loss 2522.029704
iteration 50000 / 200000: loss 2939.215247
iteration 60000 / 200000: loss 2763.794346
iteration 70000 / 200000: loss 3035.998231
iteration 80000 / 200000: loss 3004.539340
iteration 90000 / 200000: loss 3129.245537
iteration 100000 / 200000: loss 2832.070126
iteration 110000 / 200000: loss 2462.158800
iteration 120000 / 200000: loss 2883.961852
iteration 130000 / 200000: loss 2821.917343
iteration 140000 / 200000: loss 2796.670319
iteration 150000 / 200000: loss 2672.465580
iteration 160000 / 200000: loss 3015.316591
iteration 170000 / 200000: loss 2492.695250
iteration 180000 / 200000: loss 2772.264356
iteration 190000 / 200000: loss 2960.981560
MSE scikit-learn: 2912.521795117546
MSE Coordinate descent model : 2860.481173333527


In [11]:
model = LassoProximalGradientDescent(alpha=2)
model.train(X, y, learning_rate=1e-2,verbose=True, num_iters=200_000)
pred = model.predict(X)
mse= mean_squared_error(pred, y)

sk_model = Lasso(alpha=2, fit_intercept=True)
sk_model.fit(X, y)
sk_pred = sk_model.predict(X)
sk_mse = mean_squared_error(sk_pred, y)

print("MSE scikit-learn:", sk_mse)
print("MSE Coordinate descent model :", mse)
assert mse - sk_mse < 50

iteration 0 / 200000: loss 30456.151561
iteration 10000 / 200000: loss 2978.751991
iteration 20000 / 200000: loss 2624.837016
iteration 30000 / 200000: loss 2899.674633
iteration 40000 / 200000: loss 2875.927456
iteration 50000 / 200000: loss 3119.633971
iteration 60000 / 200000: loss 2840.439933
iteration 70000 / 200000: loss 2383.422342
iteration 80000 / 200000: loss 3074.118712
iteration 90000 / 200000: loss 2996.703135
iteration 100000 / 200000: loss 2740.101994
iteration 110000 / 200000: loss 2568.636583
iteration 120000 / 200000: loss 2713.078358
iteration 130000 / 200000: loss 2612.932852
iteration 140000 / 200000: loss 2780.107961
iteration 150000 / 200000: loss 3013.001392
iteration 160000 / 200000: loss 2988.083092
iteration 170000 / 200000: loss 3268.893132
iteration 180000 / 200000: loss 2886.049508
iteration 190000 / 200000: loss 2964.903220
MSE scikit-learn: 5650.287416333697
MSE Coordinate descent model : 2859.9040859512493


# Lasso Projected Gradient Descent

In [12]:
class LassoProjectedGradientDescent():
    def __init__(self,  alpha=0.1):
        self.w_p = None
        self.w_n = None
        self.b = None
        self.alpha = alpha

    def train(self, X, y, learning_rate=1e-3, num_iters=100, batch_size=200, verbose=False):
        N, d = X.shape
        
        if self.w_p is None: # Initialization
            self.w_p = 0.001 * np.random.randn(d)
            self.w_n = 0.001 * np.random.randn(d)
            self.b = 0.0

        # Run stochastic gradient descent to optimize w
        
        loss_history = []
        for it in range(num_iters):
            X_batch = None
            y_batch = None
                                                               
            # Sample batch_size elements in X_batch and y_batch
            # X_batch shape is  (batch_size, d) and y_batch shape is (batch_size,)                                                                                          
            # Hint: Use np.random.choice to generate indices
            # YOUR CODE HERE
            choice = np.random.choice(N, batch_size, replace=False)

            X_batch = X[choice, :]
            y_batch = y[choice ]
            
            # evaluate loss and gradient
            loss, dw, db = self.loss(X_batch, y_batch)

            # perform parameter update                                                                
            # Update the weights w using the gradient and the learning rate.  
            # Project for w_p and w_n
            # YOUR CODE HERE
            
            Dl = l1_subgradient(self.w_p - self.w_n)
            
            Dw = - X_batch.T @ (y_batch - X_batch @ (self.w_p - self.w_n))
            dw_n = - Dw + Dl
            dw_p = Dw + Dl
            
            self.w_p -= learning_rate * dw_p
            self.w_n -= learning_rate * dw_n
            self.b -= learning_rate * db
            
            if verbose and it % 10000 == 0:
                print("iteration %d / %d: loss %f" % (it, num_iters, loss))

    def predict(self, X):
        # YOUR CODE HERE
        w = self.w_p - self.w_n
        return (X @ w + self.b)

    def loss(self, X_batch, y_batch):
        # YOUR CODE HERE
        w = self.w_p - self.w_n
        return lasso_subgradient_mse_loss_vectorized(w, self.b, X_batch, y_batch, self.alpha)

In [13]:
model = LassoProjectedGradientDescent(alpha=0.1)
model.train(X, y, learning_rate=1e-2,verbose=True, num_iters=200_000)
pred = model.predict(X)
mse= mean_squared_error(pred, y)

sk_model = Lasso(alpha=0.1, fit_intercept=True)
sk_model.fit(X, y)
sk_pred = sk_model.predict(X)
sk_mse = mean_squared_error(sk_pred, y)

print("MSE scikit-learn:", sk_mse)
print("MSE Coordinate descent model :", mse)
assert mse - sk_mse < 50

iteration 0 / 200000: loss 28567.636904
iteration 10000 / 200000: loss 2940.553811
iteration 20000 / 200000: loss 2897.599916
iteration 30000 / 200000: loss 2808.661026
iteration 40000 / 200000: loss 2988.110346
iteration 50000 / 200000: loss 2872.691179
iteration 60000 / 200000: loss 2960.509983
iteration 70000 / 200000: loss 2838.776838
iteration 80000 / 200000: loss 2795.856593
iteration 90000 / 200000: loss 3062.493700
iteration 100000 / 200000: loss 3159.007085
iteration 110000 / 200000: loss 2856.263445
iteration 120000 / 200000: loss 2720.557311
iteration 130000 / 200000: loss 2722.399189
iteration 140000 / 200000: loss 2687.031592
iteration 150000 / 200000: loss 2868.562390
iteration 160000 / 200000: loss 3246.596662
iteration 170000 / 200000: loss 3046.540164
iteration 180000 / 200000: loss 3008.233414
iteration 190000 / 200000: loss 3071.134966
MSE scikit-learn: 2912.521795117546
MSE Coordinate descent model : 2862.5185906840416


In [14]:
model = LassoProjectedGradientDescent(alpha=2)
model.train(X, y, learning_rate=1e-2,verbose=True, num_iters=200_000)
pred = model.predict(X)
mse= mean_squared_error(pred, y)

sk_model = Lasso(alpha=2, fit_intercept=True)
sk_model.fit(X, y)
sk_pred = sk_model.predict(X)
sk_mse = mean_squared_error(sk_pred, y)

print("MSE scikit-learn:", sk_mse)
print("MSE Coordinate descent model :", mse)
assert mse - sk_mse < 50

iteration 0 / 200000: loss 29611.270699
iteration 10000 / 200000: loss 2709.374767
iteration 20000 / 200000: loss 2719.896489
iteration 30000 / 200000: loss 2586.117845
iteration 40000 / 200000: loss 2425.222663
iteration 50000 / 200000: loss 2721.072462
iteration 60000 / 200000: loss 3308.369148
iteration 70000 / 200000: loss 2769.746584
iteration 80000 / 200000: loss 2825.629316
iteration 90000 / 200000: loss 2658.294383
iteration 100000 / 200000: loss 3336.120862
iteration 110000 / 200000: loss 3060.053857
iteration 120000 / 200000: loss 2994.202067
iteration 130000 / 200000: loss 2776.462763
iteration 140000 / 200000: loss 2965.001428
iteration 150000 / 200000: loss 2945.428378
iteration 160000 / 200000: loss 2692.192247
iteration 170000 / 200000: loss 3014.965738
iteration 180000 / 200000: loss 2816.871094
iteration 190000 / 200000: loss 3096.179232
MSE scikit-learn: 5650.287416333697
MSE Coordinate descent model : 2864.6678080840775


# Lasso Coordinate Descent (no intercept)

In [15]:
class LassoCoordinateDescent():
    def __init__(self, alpha=0.1):
        self.w = None
        self.alpha = alpha

    def train(self, X, y, num_iters=1000):
        N, d = X.shape
        
        # YOUR CODE HERE
        
        # source: https://xavierbourretsicotte.github.io/lasso_implementation.html#Algorithm
        if self.w is None:
            self.w = 0.001 * np.random.randn(d)
        
        for i in range(num_iters):
            for j in range(d):
                Xj = X[:,j]
                y_pred = X @ self.w 
                self.w[j] =  soft_threshold(Xj.T @ ( y - y_pred + self.w[j] * Xj), self.alpha)

    def predict(self, X): 
        # YOUR CODE HERE
        return X @ self.w

In [16]:
model = LassoCoordinateDescent(alpha=0.1)
model.train(X, y)
pred = model.predict(X)
mse= mean_squared_error(pred, y)

sk_model = Lasso(alpha=0.1, fit_intercept=False)
sk_model.fit(X, y)
sk_pred = sk_model.predict(X)
sk_mse = mean_squared_error(sk_pred, y)

print("MSE scikit-learn:", sk_mse)
print("MSE Coordinate descent model :", mse)
assert mse - sk_mse < 50

MSE scikit-learn: 26057.118798659812
MSE Coordinate descent model : 26004.297709341427


In [17]:
model = LassoCoordinateDescent(alpha=2)
model.train(X, y)
pred = model.predict(X)
mse= mean_squared_error(pred, y)

sk_model = Lasso(alpha=2, fit_intercept=False)
sk_model.fit(X, y)
sk_pred = sk_model.predict(X)
sk_mse = mean_squared_error(sk_pred, y)

print("MSE scikit-learn:", sk_mse)
print("MSE Coordinate descent model :", mse)
assert mse - sk_mse < 50

MSE scikit-learn: 28794.88441987604
MSE Coordinate descent model : 26006.416566913213
